# ANN with AST

In [148]:
from collections import Counter

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tree_sitter_python as tspython
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from tree_sitter import Language, Parser

PY_LANGUAGE = Language(tspython.language())
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cuda')

## Utils

In [149]:
def set_seed(seed: int = 420):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def calculate_metrics_classifier(true_label, predicted):
    f1 = f1_score(true_label, predicted)
    roc_auc = roc_auc_score(true_label, predicted)
    recall = recall_score(true_label, predicted)
    accuracy = accuracy_score(true_label, predicted)
    precision = precision_score(true_label, predicted)

    print(
        f"F1: {f1} | ROC/AUC: {roc_auc} | RECALL: {recall} | PRECISION: {precision} | ACCURACY: {accuracy}"
    )
    return f1

## Data Loading

In [150]:
set_seed()
df = pd.read_csv("../../data/generated/dataset.csv")

print(f"Total size: {len(df)}\n")

train_val_df, test_df = train_test_split(
    df, test_size=0.2, stratify=df["generated"], random_state=420
)
train_df, val_df = train_test_split(
    df, test_size=0.22, stratify=df["generated"], random_state=420
)

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

Total size: 12428

Train size: 9693
Validation size: 2735
Test size: 2486


Check that the data is balanced

In [151]:
print(f"Mean generated (train): {train_df['generated'].mean()}")
print(f"Mean generated (val): {val_df['generated'].mean()}")
print(f"Mean generated (test): {test_df['generated'].mean()}")

Mean generated (train): 0.5212008666047663
Mean generated (val): 0.5210237659963437
Mean generated (test): 0.5213193885760258


## Dataset building

In [152]:
parser = Parser(PY_LANGUAGE)
node_types = set()


def walk_tree(node, types):
    types.append(node.type)
    for child in node.children:
        walk_tree(child, types)


def code_to_feature_vector(code):
    code = code.encode("utf-8")
    tree = parser.parse(code)
    types = []
    walk_tree(tree.root_node, types)
    counts = Counter(types)
    feature_vector = [counts.get(typ, 0) for typ in node_types]

    return feature_vector


# Gather all node types
for _, row in train_df.iterrows():
    tree = parser.parse(str.encode(row["code"]))
    types = []
    walk_tree(tree.root_node, types)
    node_types.update(types)

node_types = sorted(node_types)
type_to_idx = {typ: i for i, typ in enumerate(node_types)}

Save node types for inference

In [153]:
with open("../../data/ast/node_types.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(node_types))

In [154]:
def ast_encoding(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    features_df = pd.DataFrame(df["code"].apply(code_to_feature_vector).apply(pd.Series))
    features_df.columns = list(node_types)
    return features_df, df["generated"]

In [155]:
train_features, train_target = ast_encoding(train_df)
val_features, val_target = ast_encoding(val_df)
test_features, test_target = ast_encoding(test_df)

In [156]:
train_max = train_features.to_numpy().max()

In [157]:
class VectorDataset(Dataset):
    def __init__(self, features: pd.DataFrame, labels: pd.DataFrame,train_max=train_max):
        self.train_max=train_max
        self.features = torch.FloatTensor(features.to_numpy())
        self.labels = torch.LongTensor(labels.to_numpy())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx]/self.train_max, self.labels[idx]


train_dataset = VectorDataset(train_features, train_target)
val_dataset = VectorDataset(val_features, val_target)
test_dataset = VectorDataset(test_features, test_target)

In [158]:
train_batch_size = 32
eval_batch_size = 128

train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    generator=torch.Generator().manual_seed(420),
)

val_loader = DataLoader(
    val_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
)

test_loader = DataLoader(test_dataset, batch_size=eval_batch_size, shuffle=False)

## Model

In [159]:
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.LeakyReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Dropout(0.2),  
            nn.Linear(32, 16), 
            nn.LeakyReLU(),
            nn.Dropout(0.2),     
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

## Training

In [160]:
def train_model(
    train_loader: DataLoader, model: nn.Module, optimizer: optim.Optimizer, criterion
):
    model.train()
    train_losses = []
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.flatten().float(), labels.float())
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    return np.mean(train_losses)


def eval_model(eval_loader: DataLoader, model: nn.Module, criterion):
    model.eval()

    eval_losses = []
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in tqdm(eval_loader, desc="Evaluating"):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            outputs = model(inputs)
            eval_losses.append(
                criterion(outputs.flatten().float(), labels.float()).item()
            )
            preds = (outputs > 0.5).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_preds = np.array(all_preds).flatten()
    all_labels = np.array(all_labels).flatten()
    recall = calculate_metrics_classifier(all_labels, all_preds)
    return np.mean(eval_losses), recall

In [161]:
model = BinaryClassifier(input_dim=len(node_types)).to(DEVICE)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=5e-4)


best_recall = 0
epochs = 100
early_stopping = 20
es_index = 0
for epoch in range(1, epochs + 1):
    print(f"\nEpoch {epoch}")
    train_loss = train_model(train_loader, model, optimizer, criterion)
    print(f"Mean train loss: {train_loss:.3f}")
    eval_loss, recall = eval_model(val_loader, model, criterion)
    print(f"Mean val loss: {eval_loss:.3f}")

    if recall > best_recall:
        best_recall = recall
        torch.save(model.state_dict(), "../../data/generated/best_nn_ast.pth")
        es_index = 0

    else:
        es_index += 1

    if es_index > early_stopping:
        break


Epoch 1


Training: 100%|██████████| 303/303 [00:00<00:00, 515.31it/s]


Mean train loss: 0.693


Evaluating: 100%|██████████| 22/22 [00:00<00:00, 578.95it/s]


F1: 0.6850961538461539 | ROC/AUC: 0.5 | RECALL: 1.0 | PRECISION: 0.5210237659963437 | ACCURACY: 0.5210237659963437
Mean val loss: 0.692

Epoch 2


...

F1: 0.7567981616238989 | ROC/AUC: 0.7710941475826972 | RECALL: 0.6933333333333334 | PRECISION: 0.8330522765598651 | ACCURACY: 0.7678244972577697
Mean val loss: 0.509

Epoch 100


Training: 100%|██████████| 303/303 [00:00<00:00, 613.36it/s]


Mean train loss: 0.457


Evaluating: 100%|██████████| 22/22 [00:00<00:00, 611.11it/s]

F1: 0.7827042872847196 | ROC/AUC: 0.7846605062274007 | RECALL: 0.7494736842105263 | PRECISION: 0.8190184049079755 | ACCURACY: 0.7831809872029251
Mean val loss: 0.485


## Testing

In [162]:
best_model = BinaryClassifier(input_dim=len(node_types)).to(DEVICE)
best_model.load_state_dict(torch.load("../../data/generated/best_nn_ast.pth"))


eval_model(test_loader, best_model, criterion)

Evaluating: 100%|██████████| 20/20 [00:00<00:00, 487.79it/s]

F1: 0.7873267326732674 | ROC/AUC: 0.7847481585226683 | RECALL: 0.7669753086419753 | PRECISION: 0.8087876322213181 | ACCURACY: 0.7839903459372486


(0.4990876778960228, 0.7873267326732674)